In [15]:
#Import Libraries
import os
import re
import sys
import math
import time
import sys
import random
from random import shuffle
import numpy as np
import scipy.signal
import gym
from gym import spaces
from gym.spaces import Box, Discrete
from gym.utils import seeding
import tensorflow as tf
import tensorflow.compat.v1 as tf
import datetime
from collections import deque
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam

In [16]:
#Default Parameters for HPC
 
max_q_size = 256
mlp_size = 1024
wait_max_t = 14400
run_max_t = 14400
task_fea = 6
debug = False
task_seq_t = 256

In [10]:
os.chdir("D:\IUB\Course_Work\Sem-II\RL\Project\Input")

In [26]:
#Defining Machine State

def start_mach(id):
    id = id
    run_ts_id= -1
    free_now = True
    past_tasks = []
    
def task_details(past_tasks, ts_id):
    if free_now:
        run_ts_id = ts_id
        free_now = False
        past_tasks.append(ts_id)
        return True
    else:
        return False
    
def reset(run_ts_id):
    free_now = True
    run_ts_id = -1
    past_tasks = []
    
def release(free_now,run_ts_id):
    if free_now:
        return -1
    else:
        free_now = True
        run_ts_id = -1
        return 1

def id_update(id):
    return "M["+str(id)+"] "

In [28]:
#Defining Clusters

def def_clus(cluster_name, node_num, num_procs_per_node):
    clus_name = cluster_name
    total_num_node = node_num
    node_free = node_num
    node_used = 0
    procs_pnode = procs_pnode
    all_nodes = []

    for i in range(total_node):
        all_nodes.append(start_mach(i))
    return all_nodes, node_used,clus_name, total_num_node

def alloc_tas(task):
    if task.request_number_of_nodes != -1 and task.request_number_of_nodes > free_node:
        return False
    if task.request_number_of_nodes != -1 and task.request_number_of_nodes <= free_node:
        return True

def alloc(task_id, request_num_procs):
    alloc_nodes = []
    req_node = int(round(request_num_procs) / (num_procs_per_node))

    if req_node >free_node:
        return []

    alloc = 0

    for t in all_nodes:
        if allocated == req_node:
            return allocated_nodes
        if t.taken_by_task(task_id):
            allocated += 1
            nd_used += 1
            nod_free -= 1
            alloc_nodes.append(t)

    if alloc == req_node:
        return alloc_nodes

    print ("Free nodes but cant be allocated.")
    return []

    req_node = int(round((task.req_no_procs)/(num_procs_pnode)))
    task.req_no_procs = req_node
    if req_node > free_node:
        return False
    else:
        return True

def reset():
    used_nod = 0
    free_nod = all_node
    for i in all_nodes:
        i.reset()

def check_free(used_node):
    if used_node == 0:
        return True
    return False

def free_node_action(removes, used_node, free_node):
    used_node -= len(removes)
    free_node += len(removes)

    for t in removes:
        t.free_node_action()

In [32]:
def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

def placeholder(dim=None):
    return tf.placeholder(dtype=tf.float32, shape=combined_shape(None, dim))


def placeholders(*args):
    return [placeholder(dim) for dim in args]


def placeholder_from_space(space):
    if isinstance(space, Box):
        return placeholder(space.shape)
    elif isinstance(space, Discrete):
        return tf.placeholder(dtype=tf.int32, shape=(None,))
    raise NotImplementedError


def placeholders_from_spaces(*args):
    return [placeholder_from_space(space) for space in args]


def get_vars(scope=''):
    return [x for x in tf.trainable_variables() if scope in x.name]


def count_vars(scope=''):
    v = get_vars(scope)
    return sum([np.prod(var.shape.as_list()) for var in v])


def discount_cumsum(x, discount):
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

def hpc_init():
    action_space = spaces.Discrete(max_q_size)
    observation_space = spaces.Box(low=0.0, high=1.0,shape=(task_fea * max_q_size,),dtype=np.float32)

    task_q = []
    running_tasks = []
    vsbl_tasks = []
    pairs = []

    current_timestamp = 0
    start = 0
    next_arriving_task_idx = 0
    last_task_in_batch = 0
    num_task_in_batch = 0
    start_idx_last_reset = 0

    loads = None
    cluster = None

    bsld_algo_dict = {}
    scheduled_rl = {}
    penalty = 0
    pivot_task = False
    scheduled_scores = []

    enable_preworkloads = False
    pre_workloads = []

def start_hpc( workload_file='', sched_file=''):
    print("loading from dataset:", workload_file)
    loads = Workloads(workload_file)
    cluster = def_clus(loads.max_nodes, loads.max_procs / loads.max_nodes)
    penalty_task_score = task_seq_siz * loads.max_exec_time / 10

def seed(seed=None):
    np_random, seed = seeding.np_random(seed)
    return [seed]

def gen_preworkloads(size):

    running_task_size = size
    for i in range(running_task_size):
        _task = loads[start - i - 1]
        req_num_of_processors = _task.request_number_of_processors
        runtime_of_task = _task.request_time
        task_tmp = Task()
        task_tmp.task_id = (-1 - i)
        task_tmp.request_number_of_processors = req_num_of_processors
        task_tmp.run_time = runtime_of_task
        if cluster.can_allocated(task_tmp):
            running_tasks.append(task_tmp)
            task_tmp.scheduled_time = max(0, (current_timestamp - random.randint(0, max(runtime_of_task, 1))))
            task_tmp.allocated_machines = cluster.allocate(task_tmp.task_id, task_tmp.request_number_of_processors)
            pre_workloads.append(task_tmp)
        else:
            break

def refill_preworkloads():
    for _task in pre_workloads:
        running_tasks.append(_task)
        _task.allocated_machines = cluster.allocate(_task.task_id, _task.request_number_of_processors)

def reset(task_seq_siz):
    cluster.reset()
    loads.reset()

    task_queue = []
    running_tasks = []
    vsbl_tasks = []
    pairs = []

    current_timestamp = 0
    start = 0
    next_arriving_task_idx = 0
    last_task_in_batch = 0
    num_task_in_batch = 0
    scheduled_rl = {}
    penalty = 0
    pivot_task = False
    scheduled_scores = []

    pre_workloads = []

    start = np_random.randint(task_seq_siz, (loads.size() - task_seq_siz - 1))
    start_idx_last_reset = start
    num_task_in_batch = task_seq_siz
    last_task_in_batch = start + num_task_in_batch
    current_timestamp = loads[start].submit_time
    task_queue.append(loads[start])
    next_arriving_task_idx = start + 1

    if enable_preworkloads:
        gen_preworkloads(task_seq_siz + np_random.randint(task_seq_siz))

    return build_observation(), build_critic_obs()

def reset_for_test(num, start):
    cluster.reset()
    loads.reset()

    task_queue = []
    running_tasks = []
    vsbl_tasks = []
    pairs = []

    current_timestamp = 0
    start = 0
    next_arriving_task_idx = 0
    last_task_in_batch = 0
    num_task_in_batch = 0
    scheduled_rl = {}
    penalty = 0
    pivot_task = False
    scheduled_scores = []

    task_seq_siz = num

    start = np_random.randint(task_seq_siz, (loads.size() - task_seq_siz - 1))
    start_idx_last_reset = start
    num_task_in_batch = task_seq_siz
    last_task_in_batch = start + num_task_in_batch
    current_timestamp = loads[start].submit_time
    task_queue.append(loads[start])
    next_arriving_task_idx = start + 1

def moveforward_for_resources_backfill_greedy(task, scheduled_logs):
    assert not cluster.can_allocated(task)

    earliest_start_time = current_timestamp
    running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.request_time))
    free_processors = cluster.free_node * cluster.num_procs_per_node
    for running_task in running_tasks:
        free_processors += len(running_task.allocated_machines) * cluster.num_procs_per_node
        earliest_start_time = (running_task.scheduled_time + running_task.request_time)
        if free_processors >= task.request_number_of_processors:
            break

    while not cluster.can_allocated(task):

        #backfill tasks
        task_queue.sort(key=lambda _j: fcfs_score(_j))
        task_queue_iter_copy = list(task_queue)
        for _j in task_queue_iter_copy:
            if cluster.can_allocated(_j) and (current_timestamp + _j.request_time) < earliest_start_time:
                assert _j.scheduled_time == -1
                _j.scheduled_time = current_timestamp
                _j.allocated_machines = cluster.allocate(_j.task_id, _j.request_number_of_processors)
                running_tasks.append(_j)
                score = (task_score(_j) / num_task_in_batch)
                scheduled_logs[_j.task_id] = score
                task_queue.remove(_j)

        assert running_tasks
        running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.run_time))
        next_resource_release_time = (running_tasks[0].scheduled_time + running_tasks[0].run_time)
        next_resource_release_machines = running_tasks[0].allocated_machines

        if next_arriving_task_idx < last_task_in_batch \
                and loads[next_arriving_task_idx].submit_time <= next_resource_release_time:
            current_timestamp = max(current_timestamp, loads[next_arriving_task_idx].submit_time)
            task_queue.append(loads[next_arriving_task_idx])
            next_arriving_task_idx += 1
        else:
            current_timestamp = max(current_timestamp, next_resource_release_time)
            cluster.release(next_resource_release_machines)
            running_tasks.pop(0)

def schedule_curr_sequence_reset(score_fn):
    scheduled_logs = {}
    while True:
        task_queue.sort(key=lambda j: score_fn(j))
        task_for_scheduling = task_queue[0]
        if not cluster.can_allocated(task_for_scheduling):
            moveforward_for_resources_backfill_greedy(task_for_scheduling, scheduled_logs)

        assert task_for_scheduling.scheduled_time == -1
        task_for_scheduling.scheduled_time = current_timestamp
        task_for_scheduling.allocated_machines = cluster.allocate(task_for_scheduling.task_id,
                                                                      task_for_scheduling.request_number_of_processors)
        running_tasks.append(task_for_scheduling)
        score = (task_score(task_for_scheduling) / num_task_in_batch)
        scheduled_logs[task_for_scheduling.task_id] = score
        task_queue.remove(task_for_scheduling)

        not_empty = moveforward_for_task()
        if not not_empty:
            break

    cluster.reset()
    loads.reset()
    task_queue = []
    running_tasks = []
    vsbl_tasks = []
    pairs = []
    current_timestamp = loads[start].submit_time
    task_queue.append(loads[start])
    last_task_in_batch = start + num_task_in_batch
    next_arriving_task_idx = start + 1

    if enable_preworkloads:
        refill_preworkloads()

    return scheduled_logs

def build_critic_obs():
    vector = np.zeros(task_seq_siz * 3, dtype=float)
    earlist_task = loads[start_idx_last_reset]
    earlist_submit_time = earlist_task.submit_time
    pairs = []
    for i in range(start_idx_last_reset, last_task_in_batch + 1):
        task = loads[i]
        submit_time = task.submit_time - earlist_submit_time
        request_processors = task.request_number_of_processors
        request_time = task.request_time

        normalized_submit_time = min(float(submit_time) / float(wait_max_t), 1.0 - 1e-5)
        normalized_run_time = min(float(request_time) / float(loads.max_exec_time), 1.0 - 1e-5)
        normalized_request_nodes = min(float(request_processors) / float(loads.max_procs), 1.0 - 1e-5)

        pairs.append([normalized_submit_time, normalized_run_time, normalized_req_nodes])

    for i in range(task_seq_siz):
        vector[i * 3:(i + 1) * 3] = pairs[i]

    return vector

def build_observation():
    vector = np.zeros((max_q_size) * task_fea, dtype=float)
    task_queue.sort(key=lambda task: fcfs_score(task))
    vsbl_tasks = []
    for i in range(0, max_q_size):
        if i < len(task_queue):
            vsbl_tasks.append(task_queue[i])
        else:
            break
    vsbl_tasks.sort(key=lambda j: fcfs_score(j))

    vsbl_tasks = []
    if len(task_queue) <= max_q_size:
        for i in range(0, len(task_queue)):
            vsbl_tasks.append(task_queue[i])
    else:
        vsbl_random = []
        random_index = 0
        shuffled = list(task_queue)
        shuffle(shuffled)
        for i in range(0, max_q_size):
            vsbl_random.append(shuffled[i])

        index = 0

        while index < max_q_size:

            if (not random_task in vsbl_tasks) and index < max_q_size:
                vsbl_tasks.append(random_task)
                index += 1

    pairs = []
    add_skip = False
    for i in range(0, max_q_size):
        if i < len(vsbl_tasks) and i < (max_q_size):
            task = vsbl_tasks[i]
            submit_time = task.submit_time
            req_processors = task.req_no_of_prccessors
            req_time = task.request_time
            wait_time = curr_timestamp - submit_time

            nrml_wait_time = min(float(wait_time) / float(wait_max_t), 1.0 - 1e-5)
            nrml_run_time = min(float(request_time) / float(loads.max_exec_time), 1.0 - 1e-5)
            nrml_req_nodes = min(float(request_processors) / float(loads.max_procs), 1.0 - 1e-5)

            if cluster.can_allocated(task):
                can_schedule_now = 1.0 - 1e-5
            else:
                can_schedule_now = 1e-5
            pairs.append(
                [task, normalized_wait_time, normalized_run_time, normalized_request_nodes, can_schedule_now])
        else:
            pairs.append([None, 0, 1, 1, 0])

    for i in range(0, max_q_size):
        vector[i * task_fea:(i + 1) * task_fea] = pairs[i][1:]

    return vector

def moveforward_for_resources_backfill(task):
    assert not cluster.can_allocated(task)
    earliest_start_time = current_timestamp
    running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.request_time))
    free_processors = cluster.free_node * cluster.num_procs_per_node
    for running_task in running_tasks:
        free_processors += len(running_task.allocated_machines) * cluster.num_procs_per_node
        earliest_start_time = (running_task.scheduled_time + running_task.request_time)
        if free_processors >= task.request_number_of_processors:
            break

    while not cluster.can_allocated(task):
        task_queue.sort(key=lambda _j: fcfs_score(_j))
        task_queue_iter_copy = list(task_queue)
        for _j in task_queue_iter_copy:
            if cluster.can_allocated(_j) and (current_timestamp + _j.request_time) < earliest_start_time:
                assert _j.scheduled_time == -1
                _j.scheduled_time = current_timestamp
                _j.allocated_machines = cluster.allocate(_j.task_id, _j.request_number_of_processors)
                running_tasks.append(_j)
                score = (task_score(_j) / num_task_in_batch)
                scheduled_rl[_j.task_id] = score
                task_queue.remove(_j)
        assert running_tasks
        running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.run_time))
        next_resource_release_time = (running_tasks[0].scheduled_time + running_tasks[0].run_time)
        next_resource_release_machines = running_tasks[0].allocated_machines

        if next_arriving_task_idx < last_task_in_batch \
                and loads[next_arriving_task_idx].submit_time <= next_resource_release_time:
            current_timestamp = max(current_timestamp, loads[next_arriving_task_idx].submit_time)
            task_queue.append(loads[next_arriving_task_idx])
            next_arriving_task_idx += 1
        else:
            current_timestamp = max(current_timestamp, next_resource_release_time)
            cluster.release(next_resource_release_machines)
            running_tasks.pop(0)

def skip_for_resources():
    assert running_tasks
    running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.run_time))
    next_resource_release_time = (running_tasks[0].scheduled_time + running_tasks[0].run_time)
    next_resource_release_machines = running_tasks[0].allocated_machines

    if next_arriving_task_idx < last_task_in_batch and loads[
        next_arriving_task_idx].submit_time <= next_resource_release_time:
        current_timestamp = max(current_timestamp, loads[next_arriving_task_idx].submit_time)
        task_queue.append(loads[next_arriving_task_idx])
        next_arriving_task_idx += 1
    else:
        current_timestamp = max(current_timestamp, next_resource_release_time)
        cluster.release(next_resource_release_machines)
        running_tasks.pop(0)
    return False

def moveforward_for_task():
    if task_queue:
        return True
    if next_arriving_task_idx >= last_task_in_batch:
        assert not task_queue
        return False
    while not task_queue:
        if not running_tasks:
            next_resource_release_time = sys.maxsize
            next_resource_release_machines = []
        else:
            running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.run_time))
            next_resource_release_time = (running_tasks[0].scheduled_time + running_tasks[0].run_time)
            next_resource_release_machines = running_tasks[0].allocated_machines

        if loads[next_arriving_task_idx].submit_time <= next_resource_release_time:
            current_timestamp = max(current_timestamp, loads[next_arriving_task_idx].submit_time)
            task_queue.append(loads[next_arriving_task_idx])
            next_arriving_task_idx += 1
            return True
        else:
            current_timestamp = max(current_timestamp, next_resource_release_time)
            cluster.release(next_resource_release_machines)
            running_tasks.pop(0)

def task_score(task_for_scheduling):
    reward = tf.random.normal(task_for_scheduling.run_time)
    temp = (task_for_scheduling.scheduled_time - task_for_scheduling.submit_time + task_for_scheduling.run_time)/ max(task_for_scheduling.run_time, 10)
    reward = reward * max(1.0,temp )
    return reward

def has_only_one_task():
    if len(task_queue) == 1:
        return True
    else:
        return False

def skip_schedule(next_resource_release_time,running_tasks):
    next_resource_release_time = sys.maxsize
    next_resource_release_machines = []
    if running_tasks:
        running_tasks.sort(key=lambda running_task: (running_task.scheduled_time + running_task.run_time))
        next_resource_release_time = (running_tasks[0].scheduled_time + running_tasks[0].run_time)
        next_resource_release_machines = running_tasks[0].allocated_machines

    if next_arriving_task_idx >= last_task_in_batch and not running_tasks:
        if not pivot_task:
            pivot_task = True
            return False, 0
        else:
            return False, 0

    if next_arriving_task_idx < last_task_in_batch and loads[
        next_arriving_task_idx].submit_time <= next_resource_release_time:
        current_timestamp = max(current_timestamp, loads[next_arriving_task_idx].submit_time)
        task_queue.append(loads[next_arriving_task_idx])
        next_arriving_task_idx += 1
    else:
        current_timestamp = max(current_timestamp, next_resource_release_time)
        cluster.release(next_resource_release_machines)
        running_tasks.pop(0)
    return False, 0

def schedule(task_for_scheduling):
    if not cluster.can_allocated(task_for_scheduling):
        moveforward_for_resources_backfill(task_for_scheduling)

    assert task_for_scheduling.scheduled_time == -1
    task_for_scheduling.scheduled_time = current_timestamp
    task_for_scheduling.allocated_machines = cluster.allocate(task_for_scheduling.task_id,
                                                                  task_for_scheduling.request_number_of_processors)
    running_tasks.append(task_for_scheduling)
    score = (task_score(task_for_scheduling) / num_task_in_batch)
    scheduled_rl[task_for_scheduling.task_id] = score
    task_queue.remove(task_for_scheduling)

    not_empty = moveforward_for_task()

    if not_empty:
        return False
    else:
        return True

def valid(a):
    action = a[0]
    return pairs[action][0]

def step(a):
    task_for_scheduling = pairs[a][0]

    if not task_for_scheduling:
        done, _ = skip_schedule()
    else:
        task_for_scheduling = pairs[a][0]
        done = schedule(task_for_scheduling)

    if not done:
        obs = build_observation()
        return [obs, 0, False, 0]
    else:
        rl_total = sum(scheduled_rl.values())
        rwd = -rl_total
        return [None, rwd, True]

def step_for_test(a):
    task_for_scheduling = pairs[a][0]

    if not task_for_scheduling:
        done, _ = skip_schedule()
    else:
        task_for_scheduling = pairs[a][0]
        done = schedule(task_for_scheduling)

    if not done:
        obs = build_observation()
        return [obs, 0, False, None]
    else:
        rl_total = sum(scheduled_rl.values())
        return [None, rl_total, True, None]

In [41]:
HPC_env = HPC_Environment()

ddpg.train(gym_env, True, max_episodes=1000)
ddpg.test(max_episodes=1000)

In [36]:
#Loading Workloads
tasks_all = []

def workload_start(path):
    max_val = 0
    max_exec_time = 0
    min_exec_time = sys.maxsize
    max_task_id = 0
    max_group_id = 0
    max_executable_number = 0
    max_task_id = 0
    max_nodes = 0
    max_procs = 0

    with open(path) as oline:
        for line in oline:
            if line.startswith(";"):
                if line.startswith("; MaxNodes:"):
                    max_nodes = int(line.split(":")[1].strip())
                if line.startswith("; MaxProcs:"):
                    max_procs = int(line.split(":")[1].strip())
                continue

            o = start_task(line)
            if o.run_time >max_exec_time:
                max_exec_time = o.run_time
            if o.run_time < min_exec_time:
                min_exec_time = o.run_time

            all_tasks.append(o)

            if o.request_number_of_processors > max_val:
                max = t.request_number_of_processors

    if max_procs == 0:
        max_procs = max_nodes

    all_tasks.sort(key=lambda task: task.task_id)

def size(all_tasks):
    return len(all_tasks)

def reset_load(all_tasks):
    for task in all_tasks:
        task.scheduled_time = -1

def obtain_work_item(all_tasks, item):
    return all_tasks[item]

oload = workload_start('ANL-Intrepid-2009-1.swf')

In [37]:
#Load Tasks
def start_task(line="0        0      0    0   0     0    0   0  0 0  0   0   0  0  0 0 0 0"):
    line = line.strip()
    s_array = re.split("\\s+", line)
    task_id = int(s_array[0])
    submit_time = int(s_array[1])
    wait_time = int(s_array[2])
    run_time = int(s_array[3])
    no_of_alloc_proce = int(s_array[4])
    avg_cpu_time_used = float(s_array[5])
    used_memory = int(s_array[6])

    req_no_of_proc = int(s_array[7])
    no_of_alloc_proce = max(no_of_alloc_proce,req_no_of_proc)
    req_no_of_proc = no_of_alloc_proce

    req_no_of_proc = -1

    request_time = int(s_array[8])
    if request_time == -1:
        request_time = run_time

    req_mem = int(s_array[9])
    status = int(s_array[10])
    user_id = int(s_array[11])
    group_id = int(s_array[12])
    exe_no = int(s_array[13])
    q_no = int(s_array[14])

    try:
        part_no = int(s_array[15])
    except ValueError:
        part_no = 0

    proceeding_task_number = int(s_array[16])
    othink_time_from_proceeding_task = int(s_array[17])

    random_id = .submit_time

    scheduled_time = -1

    alloc_mac = None

    slurm_in_queue_time = 0
    slurm_age = 0
    slurm_task_size = 0.0
    slurm_fair = 0.0
    slurm_partition = 0
    slurm_qos = 0
    slurm_tres_cpu = 0.0

def task_q(other,task_id):
    return task_id == other.task_id

def task_str(task_id,submit_time,req_time,req_no_of_proc):
    return "t[" + str(task_id) + "]-[" + str(req_no_of_proc) + "]-[" + str(submit_time) + "]-[" + str(req_time) + "]"

def task_feat(submit_time,req_no_of_proc,req_time,user_id,group_id, exe_no,q_no):
    return [submit_time, req_no_of_proc, req_time,user_id, group_id, exe_no, q_no]

In [38]:
#DDPG
def actor(state_shape, action_dim, action_bound, action_shift, units=(400, 300)):
    state = Input(shape=state_shape)
    x = Dense(units[0], name="L0", activation='relu')(state)
    for index in range(1, len(units)):
        x = Dense(units[index], name="L{}".format(index), activation='relu')(x)

    unscaled_output = Dense(action_dim, name="Out", activation='tanh')(x)
    scalar = action_bound * np.ones(action_dim)
    output = Lambda(lambda op: op * scalar)(unscaled_output)
    if np.sum(action_shift) != 0:
        output = Lambda(lambda op: op + action_shift)(output)  # for action range not centered at zero

    model = Model(inputs=state, outputs=output)

    return model


def critic(state_shape, action_dim, units=(48, 24)):
    inputs = [Input(shape=state_shape), Input(shape=(action_dim,))]
    concat = Concatenate(axis=-1)(inputs)
    x = Dense(units[0], name="L0", activation='relu')(concat)
    for index in range(1, len(units)):
        x = Dense(units[index], name="L{}".format(index), activation='relu')(x)
    output = Dense(1, name="Out")(x)
    model = Model(inputs=inputs, outputs=output)

    return model

def update_target_weights(model, target_model, tau=0.005):
    weights = model.get_weights()
    target_weights = target_model.get_weights()
    for i in range(len(target_weights)):  # set tau% of target model to be new weights
        target_weights[i] = weights[i] * tau + target_weights[i] * (1 - tau)
    target_model.set_weights(target_weights)


def ddpg_start(env,False,False,lr_act=0.001,lr_cric=0.01,actor_units=(24, 16),critic_units=(24, 16),sig=0.15,tu=0.125,gam=0.85,
               batch_size=32,memory_cap=10000)

    state_shape = observation_space.shape  # shape of observations
    action_dim = action_space.n
    discrete = discrete
    action_bound = (env.action_space.high - env.action_space.low) / 2 if not discrete else 1.
    action_shift = (env.action_space.high + env.action_space.low) / 2 if not discrete else 0.
    use_priority = use_priority
    memory = Memory_start(capacity=memory_cap) if use_priority else deque(maxlen=memory_cap)
    if noise == 'ou':
        noise = OrnsteinUhlenbeckNoise(mu=np.zeros(.action_dim), sigma=sigma)
    else:
        noise = NormalNoise(mu=np.zeros(.action_dim), sigma=sigma)

    # Define and initialize Actor network
    actor = actor(.state_shape, .action_dim, .action_bound, .action_shift, actor_units)
    actor_target = actor(.state_shape, .action_dim, .action_bound, .action_shift, actor_units)
    actor_optimizer = Adam(learning_rate=lr_actor)
    update_target_weights(.actor, .actor_target, tau=1.)

    # Define and initialize Critic network
    critic = critic(.state_shape, .action_dim, critic_units)
    critic_target = critic(.state_shape, .action_dim, critic_units)
    critic_optimizer = Adam(learning_rate=lr_critic)
    update_target_weights(.critic, .critic_target, tau=1.)

    # Set hyperparameters
    gamma = gamma  # discount factor
    tau = tau  # target model update
    batch_size = batch_size

    # Tensorboard
    summaries = {}

def act(state, add_noise=True):
    state = np.expand_dims(state, axis=0).astype(np.float32)
    a = actor.predict(state)
    a += noise() * add_noise * action_bound
    a = tf.clip_by_value(a, -action_bound + action_shift, action_bound + action_shift)

    summaries['q_val'] =critic.predict([state, a])[0][0]

    return a

def save_model(a_fn, c_fn):
    actor.save(a_fn)
    critic.save(c_fn)

def load_actor(, a_fn):
    actor.load_weights(a_fn)
    actor_target.load_weights(a_fn)
    print(actor.summary())

def load_critic(, c_fn):
    critic.load_weights(c_fn)
    critic_target.load_weights(c_fn)
    print(critic.summary())

def remember(state, action, reward, next_state, done):
    if use_priority:
        action = np.squeeze(action)
        transition = np.hstack([state, action, reward, next_state, done])
        memory.store(transition)
    else:
        state = np.expand_dims(state, axis=0)
        next_state = np.expand_dims(next_state, axis=0)
        memory.append([state, action, reward, next_state, done])

def replay():

    if use_priority:
        tree_idx, samples, ISWeights = memory.sample(.batch_size)
        split_shape = np.cumsum([state_shape[0], action_dim, 1, state_shape[0]])
        states, actions, rewards, next_states, dones = np.hsplit(samples, split_shape)
    else:
        ISWeights = 1.0
        samples = random.sample(memory, batch_size)
        s = np.array(samples).T
        states, actions, rewards, next_states, dones = [np.vstack(s[i, :]).astype(np.float) for i in range(5)]

    next_actions = actor_target.predict(next_states)
    q_future = critic_target.predict([next_states, next_actions])
    target_qs = rewards + q_future * gamma * (1. - dones)

    # train critic
    with tf.GradientTape() as tape:
        q_values = critic([states, actions])
        td_error = q_values - target_qs
        critic_loss = tf.reduce_mean(ISWeights * tf.math.square(td_error))

    critic_grad = tape.gradient(critic_loss, .critic.trainable_variables)  # compute critic gradient
    critic_optimizer.apply_gradients(zip(critic_grad, .critic.trainable_variables))

    # update priority
    if use_priority:
        abs_errors = tf.reduce_sum(tf.abs(td_error), axis=1)
        memory.batch_update(tree_idx, abs_errors)

    # train actor
    with tf.GradientTape() as tape:
        actions = actor(states)
        actor_loss = -tf.reduce_mean(.critic([states, actions]))

    actor_grad = tape.gradient(actor_loss, .actor.trainable_variables)  # compute actor gradient
    actor_optimizer.apply_gradients(zip(actor_grad, .actor.trainable_variables))

    # tensorboard info
    summaries['critic_loss'] = critic_loss
    summaries['actor_loss'] = actor_loss

def train(max_episodes=50, max_epochs=8000, max_steps=500, save_freq=50):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs/DDPG_basic_' + current_time
    summary_writer = tf.summary.create_file_writer(train_log_dir)

    done, episode, steps, epoch, total_reward = False, 0, 0, 0, 0
    cur_state = env.reset()
    while episode < max_episodes or epoch < max_epochs:
        if done:
            episode += 1
            print("episode {}: {} total reward, {} steps, {} epochs".format(episode, total_reward, steps, epoch))

            with summary_writer.as_default():
                tf.summary.scalar('Main/episode_reward', total_reward, step=episode)
                tf.summary.scalar('Main/episode_steps', steps, step=episode)

            summary_writer.flush()
            noise.reset()

            if steps >= max_steps:
                print("episode {}, reached max steps".format(episode))
                save_model("ddpg_actor_episode{}.p".format(episode),
                                "ddpg_critic_episode{}.p".format(episode))

            done, cur_state, steps, total_reward = False, .env.reset(), 0, 0
            if episode % save_freq == 0:
                save_model("ddpg_actor_episode{}.p".format(episode),
                                "ddpg_critic_episode{}.p".format(episode))

        a = act(cur_state)  # model determine action given state
        action = np.argmax(a) if .discrete else a[0]  # post process for discrete action space
        next_state, reward, done, _ = .env.step(action)  # perform action on env

        remember(cur_state, a, reward, next_state, done)  # add to memory
        replay()  # train models through memory replay

        update_target_weights(actor, actor_target, tu)  # iterates target model
        update_target_weights(critic, critic_target, tu)

        cur_state = next_state
        total_reward += reward
        steps += 1
        epoch += 1

        # Tensorboard update
        with summary_writer.as_default():
            if len(.memory) > batch_size:
                tf.summary.scalar('Loss/actor_loss', summaries['actor_loss'], step=epoch)
                tf.summary.scalar('Loss/critic_loss', summaries['critic_loss'], step=epoch)
            tf.summary.scalar('Main/step_reward', reward, step=epoch)
            tf.summary.scalar('Stats/q_val', .summaries['q_val'], step=epoch)

        summary_writer.flush()

    save_model("ddpg_actor_final_episode{}.p".format(episode),
                    "ddpg_critic_final_episode{}.p".format(episode))

def test(epi):
    cur_state, done, rewards = env.reset(), False, 0
    while not done:
        a = act(cur_state, add_noise=False)
        action = np.argmax(a) if discrete else a[0]  # post process for discrete action space
        next_state, reward, done, _ = env.step(action)
        cur_state = next_state
        rewards += reward
        if render:
            video.append_data(env.render(mode='rgb_array'))
    return rewards

In [40]:
HPC_env = HPC_Environment()

ddpg.train(gym_env, True, max_episodes=500)
ddpg.test(max_episodes=250)

In [42]:
epsilon = 0.01 
alpha = 0.7 
beta = 0.5
beta_increment_per_sampling = 0.001
abs_err_upper = 1

def memory_init(capacity):
    tree = SumTree(capacity)
    sample_count = 0

def __len__(self):
    return sample_count

def store( transition):
    max_p = np.max(tree.tree[-tree.capacity:])
    if max_p == 0:
        max_p = abs_err_upper
    tree.add(max_p, transition) 
    sample_count = min(sample_count + 1, tree.capacity)

def sample(n):
    b_idx, b_memory, ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, tree.data[0].size)), np.empty((n, 1))
    pri_seg = tree.total_p / n
    beta = np.min([1., beta + beta_increment_per_sampling])

    a = tree.tree[-tree.capacity:]
    min_prob = np.min(a[a != 0]) / tree.total_p  
    for i in range(n):
        a, b = pri_seg * i, pri_seg * (i + 1)
        v = np.random.uniform(a, b)
        idx, p, data = tree.get_leaf(v)
        prob = p / tree.total_p
        ISWeights[i, 0] = np.power(prob/min_prob, -beta)
        b_idx[i], b_memory[i, :] = idx, data

    return b_idx, b_memory, ISWeights

def batch_update( tree_idx, abs_errors):
    abs_errors += epsilon  # convert to abs and avoid 0
    clipped_errors = np.minimum(abs_errors, abs_err_upper)
    ps = np.power(clipped_errors, alpha)
    for ti, p in zip(tree_idx, ps):
        tree.update(ti, p)

In [43]:
data_pointer = 0

def strt_SumTree(capacity):
    tree = np.zeros(2 * capacity - 1)
    data = np.zeros(capacity, dtype=object)
    return tree,data
    
def add(p, data, data_pointer):
    tree_idx = data_pointer + capacity - 1
    data[data_pointer] = data 
    update(tree_idx, p) 
    data_pointer += 1
    if data_pointer >= capacity: 
        data_pointer = 0

def update(tree_idx, p):
    change = p - tree[tree_idx]
    tree[tree_idx] = p
    while tree_idx != 0:   
        tree_idx = (tree_idx - 1) // 2
        tree[tree_idx] += change

def get_leaf(v):
    parent_idx = 0
    while True:  
        cl_idx = 2 * parent_idx + 1      
        cr_idx = cl_idx + 1
        if cl_idx >= len(tree):     
            leaf_idx = parent_idx
            break
        else:     
            if v <= tree[cl_idx]:
                parent_idx = cl_idx
            else:
                v -= tree[cl_idx]
                parent_idx = cr_idx

    data_idx = leaf_idx - capacity + 1
    return leaf_idx, tree[leaf_idx], data[data_idx]

def total_p(tree):
    return tree[0]